In [9]:
!pip install Flask pyngrok line-bot-sdk requests openai --quiet

In [10]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
openai_api_key = userdata.get('OPENAI_API_KEY')
port = 5000

In [11]:
import os
from pyngrok import ngrok

In [12]:
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://a8ce-34-23-85-150.ngrok-free.app


In [13]:
from openai import OpenAI

client = OpenAI(
    base_url='http://120.105.97.239:11434/v1/',

    api_key='ollama',
)
def get_completion(prompt):
    messages=[
        {
            'role': 'system',
            'content': '你是一個說繁體中文的助理，只會用台灣用語，不會用中國用語',
            'role': 'user',
            'content': prompt,

        }
    ]
    response = client.chat.completions.create(
        model='jcai/llama3-taide-lx-8b-chat-alpha1:f16',
        messages=messages,

    )
    return response.choices[0].message.content


In [14]:
result = get_completion("簡介東吳大學")
print(result)

東吳大學，亦稱為Soochow University，創立於1900年，為台灣歷史最悠久的私立大學之一，也是全球第24所由基督教組織創辦的大學。學校位於台灣新北市士林區及台北市中山區，以自由、平等、人文、科學為校訓。

該校最初名為「東吳大學堂」，由美國基督教會派駐中國的差會「美北長老教會」所創辦，目標是提供高等教育給中國年輕知識份子。1910年遷至現址，並在1925年更名為東吳大學。

東吳大學以商學院及法學院最為著名，在國際上獲得肯定，此外，也設有文理學院、管理學院、法律學院、社會科學院等學院，提供多元化的學系與研究所。該校在學術研究方面表現卓越，在國內外均獲得許多獎項與肯定。

東吳大學以其美麗的校園著稱，融合了中國傳統建築風格與現代建築設計，其中最具代表性的地標，就是位於淡水河畔、由知名建築師貝聿銘設計的大忠教堂。校內還有許多歷史悠久的建築，如舊圖書館、大禮堂等，見證了學校的成長與演變。

東吳大學以自由、平等為校訓，在教學與研究上秉持著開放、包容的態度，在國內外均享有盛名。該校不僅培育出許多傑出的學術界人士，也孕育了眾多成功企業家與社會賢達，是台灣高等教育體系中，深具影響力的一所大學。


In [15]:
messages = []

In [16]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            messages.append(prompt)
            prompts = " ".join(messages)
            reply_text = get_completion(prompts)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )
            messages.append(reply_text)
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )


@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/May/2024 04:03:08] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510090608518627717","quoteToken":"WEIlxjN-cWOcX7YwbkmVODbLTQaMfR54u1E-7sfSCeORBc458KOw2mkTQJp3QyMJpHxyr9ecg854le7upCTlrf_57V0eYpbRd4F_iFQkXA9pQ2TB-65US_Hhpq3_FIg-NEOY85WAljwAaEFY6r4FNg","text":"AI 簡介東吳大學"},"webhookEventId":"01HYYNVTEN9MKRV68N4Q7G9E28","deliveryContext":{"isRedelivery":false},"timestamp":1716868999386,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"017d080990ef439b86311dc418c20831","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 04:03:35] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510090666383245522","quoteToken":"FLszvZrGkejWQo4SCNX0wloDLABF0HP7ydx6vQXLj4JO1FFpDuFj-_YtOdvbkZmzTPGR0hdItzuw4THkDeApN_iJ--xYF_mQ_RpBsad0o3owrEQAjnG3KSL7-SgPDBW6OsP4YPqSw5kTn12LfQr61w","text":"AI 那城區部地址呢？"},"webhookEventId":"01HYYNWW8AGWA8E9GX5X70G83J","deliveryContext":{"isRedelivery":false},"timestamp":1716869033741,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"9ad9a7410ef54179beedef2b89657ce6","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 04:04:00] "POST / HTTP/1.1" 200 -
